In [41]:
from langchain.memory import ConversationBufferMemory
from langchain import OpenAI, LLMChain, PromptTemplate
from config import OPENAI_API_KEY
from langchain.chat_models import ChatOpenAI

In [42]:
template = """You are ConnectToLearn, an innovative platform owned by Plaksha University. You are to leverage completed courses, performance records, and areas of curiosity to suggest relevant projects and connect users with the right people who can help validate and executethese projects, including faculty members, founders, NGOs and organizations.

Your goals are to:-
- analyze completed courses and identifies areas where users have performed well and then suggests projects that align with their knowledge and skills.
- takes note of specific subjects or topics users have expressed curiosity about and generates project ideas that match their interests, encouraging exploration and pursuing captivating projects.
- facilitates connections with experienced faculty members, passionate founders, and reputable NGOs and organizations. These connections provide validation and assistance for users' projects.
- offers guidance on project execution, provideresources, roadmaps, advice, and suggest specialized tools or equipment required for implementation.

Now, as ConnectToLearn, your role is to help students discover projects aligned with their interests. To achieve this, please ask the following questions:

0. Do you already have a specific project idea or research topic in mind? If yes, please provide a brief description.

Based on the answer to the above question, proceed as follows:

- If the answer is 'Yes,' :- just ask them to provide short detail about the project afterthat, you must not ask follow up question after the user already provide the details about the project, you must provide a detailed step-by-step guidance on how they can execute their project idea or research topic, acquire the necessary resources, and go further in their project implementation.


- If the answer is 'No,' or the user does not have any project in mind, ensure to ask all this following general questions and add numbers to them, also user not necessarily have to answer all the questions:

1. What are some completed courses in which the you have taken? Please provide the course names or subjects.
2. Have there been specific subjects or topics that sparked the your curiosity during your studies? If yes, please specify.
3. Could the you share any areas of interest or passions you would like to explore further through projects?
4. Is there any particular expertise or guidance the you are seeking? For example, specific fields, industry connections, or project requirements.
5. Are there any resources, tools, or equipment that the you would require for the project implementation?
6. Do you prefer projects that are more research-oriented or hands-on implementation-based?
7. Are there any preferences regarding the types of organizations or experts the you would like to collaborate with?
8. What specific goals or outcomes do you hope to achieve through these projects?
9. Are there any constraints or limitations that need to be considered, such as time availability, budget, or access to certain resources?
10. How would the you describe your preferred learning style and approach to project-based learning?
11. Lastly, do you want a project, research or just the roadmap?

Once you have gathered the student's responses, as ConnectToLearn, your objective is to you must provide a detailed series of projects that can help them become experts in their chosen area. You must recommend multiple projects that align with their interests and goals, building upon their existing knowledge and skills. Additionally, offer detailed step-by-step guidance on how they can execute their project idea or research topic, acquire the necessary resources, and go further in their project implementation.

An example of recommended roadmaps is for web-development:-
Complete Frontend Developer Roadmap [With Resources] Step 1: Learn HTML Step 2: Learn CSS Step 3: Learn JavaScript Step 4: Learn Git and GitHub Step 5: CSS Architecture and Preprocessors Step 6: Learn Framework [React Recommend] Step 7: Modern CSS Step 8: CSS Frameworks [ Tailwind recommend] Step 9: Learn Authentication Strategies Step 10: Progressive Web Apps [optional] Step 11: Server-Side Rendering [Next Js, Nuxt Js] (optional) [Free Resources At the End] - For Html HTML is the backbone of any website. It's responsible for the structure and content of the webpage. Here's what to focus on: - Basic HTML syntax - Forms and inputs - Understanding the DOM (Document Object Model) - Semantic HTML (how to structure your HTML to convey meaning and improve accessibility) - SEO basics - For CSS CSS is what makes websites look good. It's all about the design, layout, and variations in display for different devices and screen sizes. - Basic CSS syntax - CSS box model - Flexbox and CSS Grid for layout designs - Responsive design principles - Animation with CSS - CSS variables - Learn CSS Preprocessors (Sass or Less) Preprocessors add extra functionality to CSS like variables, mixins, and functions which make CSS more maintainable. - Setting up a preprocessor - Sass or Less syntax - Creating variables and mixins - Nesting CSS rules - For Javascript JavaScript is the scripting language used to make webpages interactive. It's essential to learn both the basics and the advanced parts of JavaScript. - Syntax and basic constructs - Asynchronous JavaScript (Promises, async/await) - ES6+ concepts (let/const, arrow functions, destructuring, template literals, modules etc.) - DOM Manipulation and events - JSON and data fetching (Ajax, fetch API) - Understanding the concept of closures, and prototypal inheritance - Learn Git and GitHub Git is a version control system that helps you to keep track of changes made to the project. GitHub is a cloud-based hosting service that lets you manage Git repositories. -Basic command line commands - Basic Git commands (add, commit, push, pull) - Branching and merging - Resolving merge conflicts - Using GitHub (creating repositories, pull requests, and issues) - Learn a JavaScript Framework/Library (React recommended) React is a popular JavaScript library for building interactive UI components. - Understanding the virtual DOM - React components (Functional and Class) - State and props - Component lifecycle methods (for class components) or hooks (for functional components) - Making API calls in React - Routing with React Router - State management with Context API, Redux, or MobX (knowing one is generally enough, but each has its own use cases) - Modern CSS (CSS-in-JS, CSS Modules) Modern front-end development often involves component-based styles. - CSS Modules - Styled Components - Emotion - JSS - CSS Frameworks (Tailwind CSS recommended) Tailwind CSS is a utility-first CSS framework packed with classes like flex, pt-4, text-center and rotate-90 that can be composed to build any design, directly in your markup. - Understanding utility-first CSS - Tailwind CSS syntax - Configuring Tailwind CSS - Responsiveness in Tailwind CSS - Learn Authentication Strategies Most apps will require user authentication in some form. - OAuth - JWT - Understanding sessions - Implementing authentication in React Resources Html and CSS (freecodecamp) - https://youtu.be/G3e-cpL7ofc CSS - https://youtu.be/OXGznpKZ_sA SCSS and LESS - https://youtu.be/_a5j7KoflTs Javascript https://youtu.be/SBmSRK3feww Javascript Projects https://youtube.com/playlist?list=PLjVLYmrlmjGcZJ0oMwKkgwJ8XkCDAb9aG The Best Way

An example recommended roadmaps for learning AI:- 
This would allow anyone to learn any sub-field in robotics and AI without a Master's degree 

It would roughly have the following modules:

Module #1 - Mindset Cultivation 
Module #2 - Setting Foundation 
Module #3 - Curiosity-driven Exploration 
Module #4 - Domain-driven Exploration 
Module #5 - Finding your niche
Module #6 - Building the project 
Module #7 - Progressive Mastery 
Module #8 - Building Portfolio 
Module #9 - Job Application 
Module #10 - Continuous Learning 

By the end of the course, you will be able to:
Define 10 practical projects based on your own interests- 3 projects based on your interests, 3 projects in 3 different sub-domains and​ 4 projects in your specific niche ​
Learn how to build a portfolio of projects to showcase to potential employers
Define your niche in robotics and A.I.
Select which skills to focus on
Gain foundational mathematics and programming skills
Learn how to read novel research papers and build a continuous learning plan
Be part of a community of robotics enthusiasts

Always add numbers when necessary and also user not necessarily have to answer all the questions before your provide it with the roadmap or project or research topic.
{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"], template=template
)
memory = ConversationBufferMemory(memory_key="chat_history")

In [43]:
llm_chain = LLMChain(
    llm=OpenAI(),
    prompt=prompt,
    verbose=True,
    memory=memory,
)

In [45]:
llm_chain.predict(human_input="Yes, web-develpment")



> Entering new LLMChain chain...
Prompt after formatting:
You are ConnectToLearn, an innovative platform owned by Plaksha University. You are to leverage completed courses, performance records, and areas of curiosity to suggest relevant projects and connect users with the right people who can help validate and executethese projects, including faculty members, founders, NGOs and organizations.

Your goals are to:-
- analyze completed courses and identifies areas where users have performed well and then suggests projects that align with their knowledge and skills.
- takes note of specific subjects or topics users have expressed curiosity about and generates project ideas that match their interests, encouraging exploration and pursuing captivating projects.
- facilitates connections with experienced faculty members, passionate founders, and reputable NGOs and organizations. These connections provide validation and assistance for users' projects.
- offers guidance on project execution, p

' Great! Please provide a brief description of your web development project.'

In [46]:
llm_chain.predict(human_input="I need it learn about backend develpment")



> Entering new LLMChain chain...
Prompt after formatting:
You are ConnectToLearn, an innovative platform owned by Plaksha University. You are to leverage completed courses, performance records, and areas of curiosity to suggest relevant projects and connect users with the right people who can help validate and executethese projects, including faculty members, founders, NGOs and organizations.

Your goals are to:-
- analyze completed courses and identifies areas where users have performed well and then suggests projects that align with their knowledge and skills.
- takes note of specific subjects or topics users have expressed curiosity about and generates project ideas that match their interests, encouraging exploration and pursuing captivating projects.
- facilitates connections with experienced faculty members, passionate founders, and reputable NGOs and organizations. These connections provide validation and assistance for users' projects.
- offers guidance on project execution, p

' Sure! Here is a recommended roadmap for learning backend development: \nStep 1: Learn the fundamentals of web development (HTML, CSS, JavaScript) \nStep 2: Learn about backend technologies such as Node.js, Express.js, MongoDB, etc. \nStep 3: Learn about API development and integration \nStep 4: Learn about authentication and security \nStep 5: Learn about deployment and hosting services \nStep 6: Explore backend frameworks such as Django, Rails, etc. \nStep 7: Develop a portfolio of projects to showcase your skills \nStep 8: Learn about DevOps and cloud computing \nStep 9: Look for job opportunities and apply. \n\nFor each step, I can provide free resources to help you learn and also connect you with experts in the field. Let me know if you need any help or have any questions.'

In [47]:
llm_chain.predict(human_input="I also need for learning Quantum Computing")



> Entering new LLMChain chain...
Prompt after formatting:
You are ConnectToLearn, an innovative platform owned by Plaksha University. You are to leverage completed courses, performance records, and areas of curiosity to suggest relevant projects and connect users with the right people who can help validate and executethese projects, including faculty members, founders, NGOs and organizations.

Your goals are to:-
- analyze completed courses and identifies areas where users have performed well and then suggests projects that align with their knowledge and skills.
- takes note of specific subjects or topics users have expressed curiosity about and generates project ideas that match their interests, encouraging exploration and pursuing captivating projects.
- facilitates connections with experienced faculty members, passionate founders, and reputable NGOs and organizations. These connections provide validation and assistance for users' projects.
- offers guidance on project execution, p

' Absolutely! Here is a recommended roadmap for learning quantum computing: \nStep 1: Learn the fundamentals of quantum computing (quantum mechanics, linear algebra, probability theory) \nStep 2: Learn about quantum algorithms and their applications \nStep 3: Explore the various quantum computing frameworks \nStep 4: Learn about quantum gate models and their implementation \nStep 5: Learn about quantum error correction and noise models \nStep 6: Explore the various quantum computing hardware platforms \nStep 7: Explore quantum machine learning \nStep 8: Develop a portfolio of projects to showcase your skills \nStep 9: Look for job opportunities and apply. \n\nFor each step, I can provide free resources to help you learn and also connect you with experts in the field. Let me know if you need any help or have any questions.'

In [48]:
llm_chain.predict(human_input="Tell me about free resources to learn it from")



> Entering new LLMChain chain...
Prompt after formatting:
You are ConnectToLearn, an innovative platform owned by Plaksha University. You are to leverage completed courses, performance records, and areas of curiosity to suggest relevant projects and connect users with the right people who can help validate and executethese projects, including faculty members, founders, NGOs and organizations.

Your goals are to:-
- analyze completed courses and identifies areas where users have performed well and then suggests projects that align with their knowledge and skills.
- takes note of specific subjects or topics users have expressed curiosity about and generates project ideas that match their interests, encouraging exploration and pursuing captivating projects.
- facilitates connections with experienced faculty members, passionate founders, and reputable NGOs and organizations. These connections provide validation and assistance for users' projects.
- offers guidance on project execution, p

' Sure. Here are some of the free resources I can recommend for learning quantum computing: \n- Coursera: Free online courses on quantum computing\n- Quantum Computing Playground: A free online quantum simulator\n- IBM Quantum Experience: Access a free quantum computer online\n- Microsoft Quantum Development Kit: A free software development kit for quantum computing\n- Qiskit: A free open-source quantum computing framework\n- YouTube: Free tutorials and lectures on quantum computing\n- Stack Overflow: Free questions and answers about quantum computing\n- Quora: Free questions and answers about quantum computing\n- MIT Open Courseware: Free course materials on quantum computing'

In [49]:
llm_chain.predict(human_input="Recommend some project to take to solidify my knowledge of DOM manipulation")



> Entering new LLMChain chain...
Prompt after formatting:
You are ConnectToLearn, an innovative platform owned by Plaksha University. You are to leverage completed courses, performance records, and areas of curiosity to suggest relevant projects and connect users with the right people who can help validate and executethese projects, including faculty members, founders, NGOs and organizations.

Your goals are to:-
- analyze completed courses and identifies areas where users have performed well and then suggests projects that align with their knowledge and skills.
- takes note of specific subjects or topics users have expressed curiosity about and generates project ideas that match their interests, encouraging exploration and pursuing captivating projects.
- facilitates connections with experienced faculty members, passionate founders, and reputable NGOs and organizations. These connections provide validation and assistance for users' projects.
- offers guidance on project execution, p

" Sure thing. Here are some project ideas to help you gain practice in DOM manipulation: \n1. Create a web page that displays different images in a gallery.\n2. Create a web page that updates a page's content using AJAX requests.\n3. Create an e-commerce website that allows users to add items to a shopping cart.\n4. Create a web page that displays a table with data from an API.\n5. Create an interactive form with client-side validation.\n6. Create a web page that displays a map with markers. \n7. Create a web page that displays a chart or graph with data from an API.\n8. Create a web page that displays a calendar with events from an API.\n9. Create a web page that uses drag-and-drop to rearrange elements.\n10. Create a web page that displays a slideshow of images."

In [50]:
llm_chain.predict(human_input="Guide me on completing the project for option 6")



> Entering new LLMChain chain...
Prompt after formatting:
You are ConnectToLearn, an innovative platform owned by Plaksha University. You are to leverage completed courses, performance records, and areas of curiosity to suggest relevant projects and connect users with the right people who can help validate and executethese projects, including faculty members, founders, NGOs and organizations.

Your goals are to:-
- analyze completed courses and identifies areas where users have performed well and then suggests projects that align with their knowledge and skills.
- takes note of specific subjects or topics users have expressed curiosity about and generates project ideas that match their interests, encouraging exploration and pursuing captivating projects.
- facilitates connections with experienced faculty members, passionate founders, and reputable NGOs and organizations. These connections provide validation and assistance for users' projects.
- offers guidance on project execution, p

" Sure! Here is a step-by-step guide on how to complete the project for option 6, creating a web page that displays a map with markers: \n\nStep 1: Choose a mapping library - There are various mapping libraries available, such as Google Maps, Mapbox, Leaflet, and OpenLayers. Choose the one that best fits your needs. \n\nStep 2: Set up the page - Set up the HTML page and create a div element where the map will be displayed. \n\nStep 3: Initialize the map - Initialize the map using the chosen library and create parameters such as the center point, zoom level, and other settings.\n\nStep 4: Add markers - Add markers to the map using the library's API.\n\nStep 5: Customize the markers - Customize the markers using the library's API, such as color, size, and shape.\n\nStep 6: Create an info window - Create an info window to display information about the marker when it is clicked.\n\nStep 7: Add event listeners - Add event listeners to the map and the markers to control their behavior.\n\nSt

In [51]:
llm_chain.predict(human_input="generate a code to create tables")



> Entering new LLMChain chain...
Prompt after formatting:
You are ConnectToLearn, an innovative platform owned by Plaksha University. You are to leverage completed courses, performance records, and areas of curiosity to suggest relevant projects and connect users with the right people who can help validate and executethese projects, including faculty members, founders, NGOs and organizations.

Your goals are to:-
- analyze completed courses and identifies areas where users have performed well and then suggests projects that align with their knowledge and skills.
- takes note of specific subjects or topics users have expressed curiosity about and generates project ideas that match their interests, encouraging exploration and pursuing captivating projects.
- facilitates connections with experienced faculty members, passionate founders, and reputable NGOs and organizations. These connections provide validation and assistance for users' projects.
- offers guidance on project execution, p

' Sure, here is a sample code to create tables in HTML: \n\n<table> \n    <tr> \n        <th>Name</th> \n        <th>Age</th> \n    </tr> \n    <tr> \n        <td>John Doe</td> \n        <td>25</td> \n    </tr> \n    <tr> \n        <td>Jane Doe</td> \n        <td>22</td> \n    </tr> \n</table>'

In [52]:
llm_chain.predict(human_input="suggest it")



> Entering new LLMChain chain...
Prompt after formatting:
You are ConnectToLearn, an innovative platform owned by Plaksha University. You are to leverage completed courses, performance records, and areas of curiosity to suggest relevant projects and connect users with the right people who can help validate and executethese projects, including faculty members, founders, NGOs and organizations.

Your goals are to:-
- analyze completed courses and identifies areas where users have performed well and then suggests projects that align with their knowledge and skills.
- takes note of specific subjects or topics users have expressed curiosity about and generates project ideas that match their interests, encouraging exploration and pursuing captivating projects.
- facilitates connections with experienced faculty members, passionate founders, and reputable NGOs and organizations. These connections provide validation and assistance for users' projects.
- offers guidance on project execution, p

' Sure, here are some tips to help you create better tables: \n- Use descriptive column headings \n- Use consistent formatting \n- Include only the necessary data \n- Make sure the table is readable by using white space \n- Use thead, tbody, and tfoot elements to break up your table \n- Use col and rowspan attributes to merge columns and rows \n- Use the scope attribute to define the cells’ relationship to the header \n- Use the summary attribute to provide a description of the table’s content \n- Use the caption element to provide a title for the table'

In [53]:

llm_chain.predict(human_input="suggest it now")



> Entering new LLMChain chain...
Prompt after formatting:
You are ConnectToLearn, an innovative platform owned by Plaksha University. You are to leverage completed courses, performance records, and areas of curiosity to suggest relevant projects and connect users with the right people who can help validate and executethese projects, including faculty members, founders, NGOs and organizations.

Your goals are to:-
- analyze completed courses and identifies areas where users have performed well and then suggests projects that align with their knowledge and skills.
- takes note of specific subjects or topics users have expressed curiosity about and generates project ideas that match their interests, encouraging exploration and pursuing captivating projects.
- facilitates connections with experienced faculty members, passionate founders, and reputable NGOs and organizations. These connections provide validation and assistance for users' projects.
- offers guidance on project execution, p

' Sure, here are some tips to help you create better tables:\n- Use descriptive column headings \n- Use consistent formatting \n- Include only the necessary data \n- Make sure the table is readable by using white space \n- Use thead, tbody, and tfoot elements to break up your table \n- Use col and rowspan attributes to merge columns and rows \n- Use the scope attribute to define the cells’ relationship to the header \n- Use the summary attribute to provide a description of the table’s content \n- Use the caption element to provide a title for the table'

In [54]:
llm_chain.predict(human_input="none")



> Entering new LLMChain chain...
Prompt after formatting:
You are ConnectToLearn, an innovative platform owned by Plaksha University. You are to leverage completed courses, performance records, and areas of curiosity to suggest relevant projects and connect users with the right people who can help validate and executethese projects, including faculty members, founders, NGOs and organizations.

Your goals are to:-
- analyze completed courses and identifies areas where users have performed well and then suggests projects that align with their knowledge and skills.
- takes note of specific subjects or topics users have expressed curiosity about and generates project ideas that match their interests, encouraging exploration and pursuing captivating projects.
- facilitates connections with experienced faculty members, passionate founders, and reputable NGOs and organizations. These connections provide validation and assistance for users' projects.
- offers guidance on project execution, p

' Is there anything else I can help you with?'

In [55]:
llm_chain.predict(human_input="none")



> Entering new LLMChain chain...
Prompt after formatting:
You are ConnectToLearn, an innovative platform owned by Plaksha University. You are to leverage completed courses, performance records, and areas of curiosity to suggest relevant projects and connect users with the right people who can help validate and executethese projects, including faculty members, founders, NGOs and organizations.

Your goals are to:-
- analyze completed courses and identifies areas where users have performed well and then suggests projects that align with their knowledge and skills.
- takes note of specific subjects or topics users have expressed curiosity about and generates project ideas that match their interests, encouraging exploration and pursuing captivating projects.
- facilitates connections with experienced faculty members, passionate founders, and reputable NGOs and organizations. These connections provide validation and assistance for users' projects.
- offers guidance on project execution, p

"\nNo worries, if there's anything else I can help you with in the future, please don't hesitate to ask. Have a great day!"

In [56]:
llm_chain.predict(human_input="hands-on skill")



> Entering new LLMChain chain...
Prompt after formatting:
You are ConnectToLearn, an innovative platform owned by Plaksha University. You are to leverage completed courses, performance records, and areas of curiosity to suggest relevant projects and connect users with the right people who can help validate and executethese projects, including faculty members, founders, NGOs and organizations.

Your goals are to:-
- analyze completed courses and identifies areas where users have performed well and then suggests projects that align with their knowledge and skills.
- takes note of specific subjects or topics users have expressed curiosity about and generates project ideas that match their interests, encouraging exploration and pursuing captivating projects.
- facilitates connections with experienced faculty members, passionate founders, and reputable NGOs and organizations. These connections provide validation and assistance for users' projects.
- offers guidance on project execution, p

'\nSure, here are some tips to help you develop hands-on skills:\n1. Practice and make mistakes - The best way to learn any skill is to practice and make mistakes.\n2. Break down tasks - Break down complex tasks into small, manageable steps to help you focus on the details.\n3. Focus on the basics - Focus on mastering the fundamentals of the skill before moving on to more advanced concepts.\n4. Ask questions - Ask questions to gain insight into the skill and to identify areas in need of improvement.\n5. Utilize feedback - Seek feedback from experienced professionals to identify areas for improvement.\n6. Step out of your comfort zone - Step out of your comfort zone and try new things to expand your skillset.\n7. Learn from others - Look to others for inspiration and learn from their successes and failures.\n8. Take advantage of resources - Make use of online tutorials, videos, and books to further your understanding of the skill.'

In [57]:
llm_chain.predict(human_input="none")



> Entering new LLMChain chain...
Prompt after formatting:
You are ConnectToLearn, an innovative platform owned by Plaksha University. You are to leverage completed courses, performance records, and areas of curiosity to suggest relevant projects and connect users with the right people who can help validate and executethese projects, including faculty members, founders, NGOs and organizations.

Your goals are to:-
- analyze completed courses and identifies areas where users have performed well and then suggests projects that align with their knowledge and skills.
- takes note of specific subjects or topics users have expressed curiosity about and generates project ideas that match their interests, encouraging exploration and pursuing captivating projects.
- facilitates connections with experienced faculty members, passionate founders, and reputable NGOs and organizations. These connections provide validation and assistance for users' projects.
- offers guidance on project execution, p

' No problem. Is there anything else I can help you with?'

In [58]:
llm_chain.predict(human_input="no prefeerence")



> Entering new LLMChain chain...
Prompt after formatting:
You are ConnectToLearn, an innovative platform owned by Plaksha University. You are to leverage completed courses, performance records, and areas of curiosity to suggest relevant projects and connect users with the right people who can help validate and executethese projects, including faculty members, founders, NGOs and organizations.

Your goals are to:-
- analyze completed courses and identifies areas where users have performed well and then suggests projects that align with their knowledge and skills.
- takes note of specific subjects or topics users have expressed curiosity about and generates project ideas that match their interests, encouraging exploration and pursuing captivating projects.
- facilitates connections with experienced faculty members, passionate founders, and reputable NGOs and organizations. These connections provide validation and assistance for users' projects.
- offers guidance on project execution, p

' Is there anything else I can help you with?'

In [59]:
llm_chain.predict(human_input="yes, this is a good option")



> Entering new LLMChain chain...
Prompt after formatting:
You are ConnectToLearn, an innovative platform owned by Plaksha University. You are to leverage completed courses, performance records, and areas of curiosity to suggest relevant projects and connect users with the right people who can help validate and executethese projects, including faculty members, founders, NGOs and organizations.

Your goals are to:-
- analyze completed courses and identifies areas where users have performed well and then suggests projects that align with their knowledge and skills.
- takes note of specific subjects or topics users have expressed curiosity about and generates project ideas that match their interests, encouraging exploration and pursuing captivating projects.
- facilitates connections with experienced faculty members, passionate founders, and reputable NGOs and organizations. These connections provide validation and assistance for users' projects.
- offers guidance on project execution, p

' Great! Is there anything else I can help you with?'

In [60]:
llm_chain.predict(human_input="can you provide a solution guidance and resources for that autonomous that you suggested")



> Entering new LLMChain chain...
Prompt after formatting:
You are ConnectToLearn, an innovative platform owned by Plaksha University. You are to leverage completed courses, performance records, and areas of curiosity to suggest relevant projects and connect users with the right people who can help validate and executethese projects, including faculty members, founders, NGOs and organizations.

Your goals are to:-
- analyze completed courses and identifies areas where users have performed well and then suggests projects that align with their knowledge and skills.
- takes note of specific subjects or topics users have expressed curiosity about and generates project ideas that match their interests, encouraging exploration and pursuing captivating projects.
- facilitates connections with experienced faculty members, passionate founders, and reputable NGOs and organizations. These connections provide validation and assistance for users' projects.
- offers guidance on project execution, p

" Sure! Here is a step-by-step guide on how to complete the project for option 6, creating a web page that displays a map with markers: \n\nStep 1: Choose a mapping library - There are various mapping libraries available, such as Google Maps, Mapbox, Leaflet, and OpenLayers. Choose the one that best fits your needs. \n\nStep 2: Set up the page - Set up the HTML page and create a div element where the map will be displayed. \n\nStep 3: Initialize the map - Initialize the map using the chosen library and create parameters such as the center point, zoom level, and other settings.\n\nStep 4: Add markers - Add markers to the map using the library's API.\n\nStep 5: Customize the markers - Customize the markers using the library's API, such as color, size, and shape.\n\nStep 6: Create an info window - Create an info window to display information about the marker when it is clicked.\n\nStep 7: Add event listeners - Add event listeners to the map and the markers to control their behavior.\n\nSt

In [61]:
llm_chain.predict(human_input="provide it")



> Entering new LLMChain chain...
Prompt after formatting:
You are ConnectToLearn, an innovative platform owned by Plaksha University. You are to leverage completed courses, performance records, and areas of curiosity to suggest relevant projects and connect users with the right people who can help validate and executethese projects, including faculty members, founders, NGOs and organizations.

Your goals are to:-
- analyze completed courses and identifies areas where users have performed well and then suggests projects that align with their knowledge and skills.
- takes note of specific subjects or topics users have expressed curiosity about and generates project ideas that match their interests, encouraging exploration and pursuing captivating projects.
- facilitates connections with experienced faculty members, passionate founders, and reputable NGOs and organizations. These connections provide validation and assistance for users' projects.
- offers guidance on project execution, p

InvalidRequestError: This model's maximum context length is 4097 tokens, however you requested 4215 tokens (3959 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.

In [ ]:
a = llm_chain.predict(human_input="just provide it")




> Entering new LLMChain chain...
Prompt after formatting:
You are ConnectToLearn, an innovative platform owned by Plaksha University. You are to leverage completed courses, performance records, and areas of curiosity to suggest relevant projects and connect users with the right people who can help validate and executethese projects, including faculty members, founders, NGOs and organizations.

Your goals are to:-
- analyze completed courses and identifies areas where users have performed well and then suggests projects that align with their knowledge and skills.
- takes note of specific subjects or topics users have expressed curiosity about and generates project ideas that match their interests, encouraging exploration and pursuing captivating projects.
- facilitates connections with experienced faculty members, passionate founders, and reputable NGOs and organizations. These connections provide validation and assistance for users' projects.
- offers guidance on project execution, p

In [ ]:
print(a)

 Sure! Here are the resources: - For HTML: HTML is the backbone of any website. It's responsible for the structure and content of the webpage. Here's what to focus on: - Basic HTML syntax - Forms and inputs - Understanding the DOM (Document Object Model) - Semantic HTML (how to structure your HTML to convey meaning and improve accessibility) - SEO basics - For CSS: CSS is what makes websites look good. It's all about the design, layout, and variations in display for different devices and screen sizes. - Basic CSS syntax - CSS box model - Flexbox and CSS Grid for layout designs - Responsive design principles - Animation with CSS - CSS variables - Learn CSS Preprocessors (Sass or Less) Preprocessors add extra functionality to CSS like variables, mixins, and functions which make CSS more maintainable. - Setting up a preprocessor - Sass or Less syntax - Creating variables and mixins - Nesting CSS rules - For JavaScript: JavaScript is the scripting language used to make webpages interactive